In [2]:
import sys
import os
import numpy as np
import traceback

import sys 
sys.path.append("..")


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass 
from physics.fom import sensitivity, sfm, mcf, tsf 
from physics.reflectometry import _kz, reflectivity
from solvers.grid_search import GridSearchSolver
from problems.base1 import Bounds, SOISpec, CapSpec, MRL, Materials, Base1OptimizationProblem, SubstrateSpec, MRL
from data.materials_loader import load_base1_materials

In [4]:
# simple magnetic SLD model for now – tweak later if you want
def m_sld_from_x(x: float) -> float:
    return 1.0 * x  # placeholder linear scaling

materials = load_base1_materials("data.json", m_sld_from_x=m_sld_from_x)

sois = [
    SOISpec("SOI1", rho_n=2.0, thickness=100.0, sigma=3.0),
    SOISpec("SOI2", rho_n=1.0, thickness=80.0, sigma=3.0),
    SOISpec("SOI2", rho_n=3.0, thickness=100.0, sigma=3.0)
]

Q = np.linspace(0.005, 0.3, 400)

bounds_x = Bounds(0.0, 1)      # x_coti
bounds_d = Bounds(20.0, 300.0)   # d_mrl
bounds_cap = Bounds(5.0, 200.0)    # d_cap

# define problem: 

problem = Base1OptimizationProblem(
    materials=materials,
    soi_list=sois,
    q_grid=Q,
    bounds_x=bounds_x,
    bounds_d=bounds_d,
    bounds_cap=bounds_cap,
)


In [5]:
space = problem.search_space
print(f"Params name: {space.names}")
print(f"dimensions: {len(space)}")

theta_raw = space.sample(5)   # (5, 3)
theta = np.vstack([space.clip(row) for row in theta_raw])  # clip each row

print("Raw samples:\n", theta_raw)
print("Clipped samples:\n", theta)

x0 = space.unpack(theta[0])   # first sample as dict
x0

print("---------------------------")
solver = GridSearchSolver(problem, n_points=100)
res = solver.run(evals=10_000)
print(res)
print("----------------------------")

Params name: ['x_coti', 'd_mrl', 'd_cap', 'cap']
dimensions: 4
Raw samples:
 [[7.82410907e-01 3.94715686e+01 9.21923770e+00 1.00000000e+00]
 [2.63762813e-01 2.73236749e+01 1.90042143e+02 0.00000000e+00]
 [6.04110243e-02 2.58564855e+02 7.95599184e+01 1.00000000e+00]
 [6.95538679e-01 1.77355720e+02 4.88751014e+01 0.00000000e+00]
 [5.18503400e-01 8.01147401e+01 1.97068999e+02 2.00000000e+00]]
Clipped samples:
 [[7.82410907e-01 3.94715686e+01 9.21923770e+00 1.00000000e+00]
 [2.63762813e-01 2.73236749e+01 1.90042143e+02 0.00000000e+00]
 [6.04110243e-02 2.58564855e+02 7.95599184e+01 1.00000000e+00]
 [6.95538679e-01 1.77355720e+02 4.88751014e+01 0.00000000e+00]
 [5.18503400e-01 8.01147401e+01 1.97068999e+02 2.00000000e+00]]
---------------------------
=== RunResults ===
n_evals : 10000
y_best  : 0.0
x_best  :
  x_coti     = 0.0
  d_mrl      = 20.0
  d_cap      = 5.0
  cap        = Al2O3
history : 10000 entries
history (first/last):
  [0] y=0.0 x={'x_coti': 0.0, 'd_mrl': 20.0, 'd_cap': 5.0, 'c